Data Load into dataframe

In [1]:
import pandas as pd
df = pd.read_csv('data/Data_Train.csv')
df.head()

,ID,Asst_Reg,GGGrade,Experience,Validation,Yearly_Income,Home_Status,Unpaid_2_years,Already_Defaulted,Designation,...,File_Status,State,Account_Open,Total_Unpaid_CL,Duration,Unpaid_Amount,Reason,Claim_Type,Due_Fee,Default
0,95074,421802,II,>10yrs,Vfied,633600.00,MORTGAGE,0,0,GLENDALE NISSAN,...,fully paid,California,17,58598.0,3 years,31216.05,debt consolidation,I,0.0,0
1,134131,3964312,IV,7yrs,Source Verified,85483.20,RENT,0,0,Business Teacher,...,whole,NC,15,18924.0,5 years,11660.49,debt consolidation,I,0.0,0
2,92602,4247560,III,<1yr,Source Verified,79200.00,RENT,0,0,driver,...,whole,Florida,7,15604.0,5 years,5637.87,major purchase,I,0.0,0
3,22591,197179,III,<1yr,Vfied,61600.00,RENT,0,0,AvePoint,...,fully paid,NewJersey,9,22410.0,3 years,15607.17,major purchase,I,0.0,1
4,125060,4646684,V,2yrs,Source Verified,68053.92,RENT,0,0,Lead Tester,...,fully paid,LA,10,36022.0,5 years,27472.86,debt consolidation,I,0.0,0


In [2]:
# encode GGGrade from roman numerals to integers and drop the original column
df['GGGrade'] = df['GGGrade'].map({'I':1, 'II':2, 'III':3, 'IV':4, 'V':5})
# set designation to lowercase
df['Designation'] = df['Designation'].str.lower()

df.head()


,ID,Asst_Reg,GGGrade,Experience,Validation,Yearly_Income,Home_Status,Unpaid_2_years,Already_Defaulted,Designation,...,File_Status,State,Account_Open,Total_Unpaid_CL,Duration,Unpaid_Amount,Reason,Claim_Type,Due_Fee,Default
0,95074,421802,2.0,>10yrs,Vfied,633600.00,MORTGAGE,0,0,glendale nissan,...,fully paid,California,17,58598.0,3 years,31216.05,debt consolidation,I,0.0,0
1,134131,3964312,4.0,7yrs,Source Verified,85483.20,RENT,0,0,business teacher,...,whole,NC,15,18924.0,5 years,11660.49,debt consolidation,I,0.0,0
2,92602,4247560,3.0,<1yr,Source Verified,79200.00,RENT,0,0,driver,...,whole,Florida,7,15604.0,5 years,5637.87,major purchase,I,0.0,0
3,22591,197179,3.0,<1yr,Vfied,61600.00,RENT,0,0,avepoint,...,fully paid,NewJersey,9,22410.0,3 years,15607.17,major purchase,I,0.0,1
4,125060,4646684,5.0,2yrs,Source Verified,68053.92,RENT,0,0,lead tester,...,fully paid,LA,10,36022.0,5 years,27472.86,debt consolidation,I,0.0,0


In [7]:
# print unique values for each column
for col in df.columns:
    print(col, df[col].unique())

ID [ 95074 134131  92602 ... 102153 115343 104968]
Asst_Reg [ 421802 3964312 4247560 ... 2624710 5966919 6637289]
GGGrade [ 2.  4.  3.  5. nan  1.]
Experience ['>10yrs' '7yrs' '<1yr' '2yrs' '1yrs' '4yrs' '9yrs' '6yrs' '5yrs' '8yrs'
 '3yrs']
Validation ['Vfied' 'Source Verified' 'Not Vfied']
Yearly_Income [633600.    85483.2   79200.   ... 510398.24 154191.84  90423.52]
Home_Status ['MORTGAGE' 'RENT' 'OWN' 'OTHER' 'NONE']
Unpaid_2_years [ 0  4  2  1  3 11  5  6  7  9 12  8 10 18 16 15 13 14]
Already_Defaulted [0 1 2 6 5 4 3]
Designation ['glendale nissan' 'business teacher' 'driver' ... 'engineer support ii'
 'interface coordinator' 'parts clerk']
Debt_to_Income [12.5496  9.0048 24.9144 ... 65.7216  0.4032 63.6888]
Postal_Code [91300. 27400. 32800.  7000. 70800. 77000. 92100.  6000. 92500. 78500.
 95900. 49400. 97200. 72200. 11900. 91900. 59100. 28300. 80800. 72900.
 45000. 27500. 87100. 55000. 30300.    nan 11700. 75200. 20800. 32500.
 35400. 28000. 29700. 17700. 95800. 85700. 84400. 1

In [10]:
# print the average yearly income of people who are in default
print(df[df['Default'] == 1]['Yearly_Income'].mean())

134271.61678584345


In [4]:
from sklearn.preprocessing import LabelEncoder

# One-hot encoding for low cardinality categorical variables
one_hot_columns = ['Home_Status', 'File_Status', 'Claim_Type', 'Validation', 'Duration']
df_encoded = pd.get_dummies(df, columns=one_hot_columns)

# Label encoding for high cardinality or ordinal categorical variables
label_columns = ['Asst_Reg', 'Designation', 'State', 'Reason', 'Sub_GGGrade']
label_encoder = LabelEncoder()

for col in label_columns:
    df_encoded[col + '_encoded'] = label_encoder.fit_transform(df_encoded[col].astype(str))
    
# Drop original categorical columns after encoding
df_encoded = df_encoded.drop(columns=label_columns)

# Display the first few rows of the encoded DataFrame
print(df_encoded.head())

# Get information about the encoded DataFrame
print(df_encoded.info())


       ID  GGGrade Experience  Yearly_Income  Unpaid_2_years  \
0   95074      2.0     >10yrs      633600.00               0   
1  134131      4.0       7yrs       85483.20               0   
2   92602      3.0       <1yr       79200.00               0   
3   22591      3.0       <1yr       61600.00               0   
4  125060      5.0       2yrs       68053.92               0   

   Already_Defaulted  Debt_to_Income  Postal_Code  Lend_Amount  \
0                  0         12.5496      91300.0     42023.25   
1                  0          9.0048      27400.0     38133.00   
2                  0         24.9144      32800.0     17100.00   
3                  0         16.5816       7000.0      5130.00   
4                  0         40.0512      70800.0     19665.00   

   Deprecatory_Records  ...  Validation_Not Vfied  Validation_Source Verified  \
0                    0  ...                 False                       False   
1                    0  ...                 False       

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV, train_test_split

# # Separate features and target
X = df_encoded.drop('Default', axis=1)
y = df_encoded['Default']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define classifiers and their parameter grids
classifiers = {
    'Logistic Regression': (LogisticRegression(), {
        'C': [0.001, 0.01, 0.1, 1, 10, 100],
        'penalty': ['l1', 'l2']
    }),
    'Decision Tree': (DecisionTreeClassifier(), {
        'max_depth': [3, 5, 10, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }),
    'Random Forest': (RandomForestClassifier(), {
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 5, 10, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }),
    'SVM': (SVC(), {
        'C': [0.1, 1, 10],
        'kernel': ['rbf', 'linear'],
        'gamma': ['scale', 'auto']
    })
}

# Perform GridSearch for each classifier
results = {}

for name, (classifier, param_grid) in classifiers.items():
    print(f"Running GridSearch for {name}...")
    grid_search = GridSearchCV(classifier, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_train_scaled, y_train)
    
    # Store results
    results[name] = {
        'best_params': grid_search.best_params_,
        'best_score': grid_search.best_score_,
        'test_accuracy': accuracy_score(y_test, grid_search.predict(X_test_scaled)),
        'classification_report': classification_report(y_test, grid_search.predict(X_test_scaled))
    }

# Print results
for name, result in results.items():
    print(f"\nResults for {name}:")
    print(f"Best Parameters: {result['best_params']}")
    print(f"Best Cross-Validation Score: {result['best_score']:.4f}")
    print(f"Test Accuracy: {result['test_accuracy']:.4f}")
    print("Classification Report:")
    print(result['classification_report'])

# Find the best overall model
best_model = max(results, key=lambda x: results[x]['test_accuracy'])
print(f"\nBest overall model: {best_model}")
print(f"Best overall test accuracy: {results[best_model]['test_accuracy']:.4f}")